In [ ]:
!pip install sklearn_crfsuite
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install tensorflow==1.15.0 keras==2.2.4
!pip3 install glove-python-binary
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 747kB 2.9MB/s 
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-2ges4j6r
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-2ges4j6r
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=17cb2bb4e74c1f2e5e2c046069542e1db6d941511b22f76daf608665a1e8efff
  Stored in directory: /tmp/pip-ephem-wheel-cache-ly9rve76/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 317kB 36.8MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 34.6MB/s 
     |████████████████████████████████| 512kB 31.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=96d6b92dc96cd0014afb389124263c3558a78a1e2a1fad1ba6369a8cb

In [ ]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 90.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 68.3MB/s]


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import datetime
import os
import numpy as np

from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf

import gensim.downloader as api

In [ ]:
df = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

all_dfs = [df, train, test]

In [ ]:
pretrained_w2v = api.load("word2vec-ruscorpora-300")
pretrained_w2v.save_word2vec_format('tmp')

In [ ]:
def add_suffix(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in pretrained_w2v.vocab.keys():
                res.append(w + suffix)
                has_found = True
                break
        if not has_found:
            res.append(w)
    return res

for d in all_dfs:
    d['tokens_with_suffix'] = d.apply(add_suffix, axis=1)

In [ ]:
from gensim.models import Word2Vec

w2v = Word2Vec(sentences=df['tokens_with_suffix'], size=300, window=15, min_count=2)

In [ ]:
w2v.intersect_word2vec_format('tmp')

In [ ]:
all_tags = set()
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
label_list = {i: t for i, t in enumerate(all_tags)}
labels = list(tag_to_idx.keys())
labels.remove('O')
labels = sorted(labels, key=lambda name: (name[1:], name[0]))

In [ ]:
def find_token(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ', '']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in w2v.wv.vocab.keys():
                res.append(w2v.wv.vocab[w + suffix].index)
                has_found = True
                break
        if not has_found:
            res.append(0)
    return res

for d in all_dfs:
    d['encoded_ner_tags'] = d.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
    d['int_tokens'] = d.apply(find_token, axis=1)

In [ ]:
df

,id,tokens,ner_tags,tokens_with_suffix,encoded_ner_tags,int_tokens
0,SV990125172_ОРГ__INM_18'09'58_1.pdf,"[l, *, универсальный, приложение, №, 1, постан...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-D...","[l, *, универсальный_ADJ, приложение_NOUN, №, ...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 66, 494, 64, 1, 0, 83, 59, 94, 2904, 441, ..."
1,SS528581_ОРГ__INM_17'05'39_A.pdf,"[l, лист, согласования, стр., 1, 1, 2, 9, *, 0...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[l, лист_NOUN, согласования, стр., 1, 1, 2, 9,...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 47, 35, 193, 0, 0, 4, 41, 66, 11, 17, 113,..."
2,SF435712_ОРГ__INM_15'25'41_A.pdf,"[l, приложение, №, постановлению, правительств...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, B-DOCDAT...","[l, приложение_NOUN, №, постановлению, правите...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 7, 6, 13,...","[3, 64, 1, 83, 59, 290, 1752, 74, 1, 0, 133, 2..."
3,SV990124943_ОРГ__INM_19'00'43_A.pdf,"[l, акционерное, общество, регистраторское, об...","[O, O, O, O, O, B-DOCCPTY, O, O, O, O, O, O, O...","[l, акционерное, общество_NOUN, регистраторско...","[15, 15, 15, 15, 15, 3, 15, 15, 15, 15, 15, 15...","[3, 112, 43, 6621, 43, 1315, 0, 56, 80, 0, 189..."
4,S29013419_ОРИГИНАЛ__IND_1_22.05.12'21'36.pdf,"[l, *, 000стандарт, безопасности, адрес:, 1973...","[O, O, O, B-DOCCPTY, O, O, O, O, O, O, O, O, O...","[l, *, 000стандарт, безопасности, адрес:, 1973...","[15, 15, 15, 3, 15, 15, 15, 15, 15, 15, 15, 15...","[3, 66, 14721, 4486, 28, 2677, 277, 1137, 8911..."
...,...,...,...,...,...,...
3995,SV990121575_ОРГ__INM_18'45'08_3.pdf,"[l, экземпляр, аостс, стс, отчет, роялти, 2, к...","[O, O, O, O, O, O, O, O, O, O, O, B-DOCAGRNUM,...","[l, экземпляр_NOUN, аостс, стс_NOUN, отчет_NOU...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 361, 8542, 418, 970, 1293, 4, 1828, 1537, ..."
3996,SV990121572_ОРГ__INM_18'45'08_3.pdf,"[l, ао, стс, отчет, роялти, 3, квартал, лиценз...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[l, ао_NOUN, стс_NOUN, отчет_NOUN, роялти_NOUN...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 67, 418, 970, 1293, 14, 1828, 1537, 93, 1,..."
3997,SV990121520_ОРГ__INM_16'31'06_3.pdf,"[l, экземпляр, ао, стс, ао, стс, отчет, роялти...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-D...","[l, экземпляр_NOUN, ао_NOUN, стс_NOUN, ао_NOUN...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 361, 67, 418, 67, 418, 970, 1293, 14, 1787..."
3998,SV990121573_ОРГ__INM_18'45'08_3.pdf,"[l, экземпляр, аостс, стс, отчет, п, роялти, 4...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[l, экземпляр_NOUN, аостс, стс_NOUN, отчет_NOU...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[3, 361, 8542, 418, 970, 19, 1293, 17, 0, 16, ..."


In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def focal_loss(gamma=2., alpha=1.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

## PRETRAINED W2V(not fixed)

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 943, 300)          7440900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_2 (CRF)                  (None, 943, 16)           560       
Total params: 7,765,476
Trainable params: 7,765,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 42s 15ms/step - loss: 0.7843 - crf_marginal_accuracy: 0.2648 - val_loss: 0.6091 - val_crf_marginal_accuracy: 0.8945
Epoch 2/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.5570 - crf_marginal_accuracy: 0.4671 - val_loss: 0.4561 - val_crf_marginal_accuracy: 0.1528
Epoch 3/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.4077 - crf_marginal_accuracy: 0.7324 - val_loss: 0.3206 - val_crf_marginal_accuracy: 0.9415
Epoch 4/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.2823 - crf_marginal_accuracy: 0.9513 - val_loss: 0.2279 - val_crf_marginal_accuracy: 0.9590
Epoch 5/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.2018 - crf_marginal_accuracy: 0.9687 - val_loss: 0.1798 - val_crf_marginal_accuracy: 0.9683
Epoch 6/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.1508 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.6794    0.5933    0.6335       150
 I-DOCAGRDATE     0.6455    0.8068    0.7172        88
  B-DOCAGRNUM     0.6899    0.6089    0.6469       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.7548    0.5350    0.6262      1243
  I-DOCAMOUNT     0.7153    0.5065    0.5930       620
    B-DOCCPTY     0.8613    0.8874    0.8742       595
    I-DOCCPTY     0.8724    0.8972    0.8846       564
 B-DOCCPTYINN     0.8170    0.9184    0.8648       282
B-DOCCUSTOMER     0.9111    0.8916    0.9012       609
I-DOCCUSTOMER     0.9326    0.9286    0.9306       462
    B-DOCDATE     0.8528    0.8264    0.8394       645
    I-DOCDATE     0.8561    0.9636    0.9067       605
     B-DOCNUM     0.8995    0.6052    0.7236       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.8331    0.7497    0.7892      6700
    macro avg     0.6992    0.6646    0.6761      6700
 weighte

## PRETRAINED W2V(fixed)

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 943, 300)          7440900   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_3 (CRF)                  (None, 943, 16)           560       
Total params: 7,765,476
Trainable params: 324,576
Non-trainable params: 7,440,900
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 40s 14ms/step - loss: 0.7059 - crf_marginal_accuracy: 0.0202 - val_loss: 0.4837 - val_crf_marginal_accuracy: 0.8589
Epoch 2/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.4329 - crf_marginal_accuracy: 0.7772 - val_loss: 0.3297 - val_crf_marginal_accuracy: 0.9113
Epoch 3/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.3150 - crf_marginal_accuracy: 0.9315 - val_loss: 0.2545 - val_crf_marginal_accuracy: 0.9360
Epoch 4/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.2577 - crf_marginal_accuracy: 0.9472 - val_loss: 0.2153 - val_crf_marginal_accuracy: 0.9522
Epoch 5/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.2233 - crf_marginal_accuracy: 0.9555 - val_loss: 0.1925 - val_crf_marginal_accuracy: 0.9511
Epoch 6/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.1989 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.3111    0.5600    0.4000       150
 I-DOCAGRDATE     0.5426    0.7955    0.6452        88
  B-DOCAGRNUM     0.2834    0.6760    0.3993       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.3086    0.7385    0.4353      1243
  I-DOCAMOUNT     0.4029    0.8565    0.5480       620
    B-DOCCPTY     0.4643    0.8420    0.5986       595
    I-DOCCPTY     0.6881    0.8528    0.7617       564
 B-DOCCPTYINN     0.5408    0.8936    0.6738       282
B-DOCCUSTOMER     0.7301    0.8752    0.7961       609
I-DOCCUSTOMER     0.7712    0.9264    0.8417       462
    B-DOCDATE     0.5693    0.7705    0.6548       645
    I-DOCDATE     0.7568    0.9719    0.8509       605
     B-DOCNUM     0.6560    0.6298    0.6426       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.4957    0.8081    0.6144      6700
    macro avg     0.4683    0.6926    0.5499      6700
 weighte

## W2V(not fixed, window=5, min_count=2) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300000, window=5, min_count=2)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 943, 50)           1240150   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_5 (CRF)                  (None, 943, 16)           560       
Total params: 1,364,726
Trainable params: 1,364,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 43s 15ms/step - loss: 0.7996 - crf_marginal_accuracy: 0.5780 - val_loss: 0.6023 - val_crf_marginal_accuracy: 0.9577
Epoch 2/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.5411 - crf_marginal_accuracy: 0.9492 - val_loss: 0.4425 - val_crf_marginal_accuracy: 0.9568
Epoch 3/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.4049 - crf_marginal_accuracy: 0.9545 - val_loss: 0.3401 - val_crf_marginal_accuracy: 0.9477


In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.0000    0.0000    0.0000       150
 I-DOCAGRDATE     0.0000    0.0000    0.0000        88
  B-DOCAGRNUM     0.0008    0.0056    0.0013       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.0615    0.8294    0.1145      1243
  I-DOCAMOUNT     0.0000    0.0000    0.0000       620
    B-DOCCPTY     0.0642    0.6286    0.1165       595
    I-DOCCPTY     0.0000    0.0000    0.0000       564
 B-DOCCPTYINN     0.0000    0.0000    0.0000       282
B-DOCCUSTOMER     0.0481    0.1839    0.0763       609
I-DOCCUSTOMER     0.8505    0.3571    0.5030       462
    B-DOCDATE     0.1011    0.4620    0.1659       645
    I-DOCDATE     0.3557    0.5686    0.4377       605
     B-DOCNUM     0.4756    0.3886    0.4277       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.0834    0.3848    0.1371      6700
    macro avg     0.1305    0.2283    0.1229      6700
 weighte

## W2V(fixed, window=5, min_count=2) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 943, 50)           1240150   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_6 (CRF)                  (None, 943, 16)           560       
Total params: 1,364,726
Trainable params: 124,576
Non-trainable params: 1,240,150
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 42s 15ms/step - loss: 0.8119 - crf_marginal_accuracy: 0.1154 - val_loss: 0.6221 - val_crf_marginal_accuracy: 0.8025
Epoch 2/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.5348 - crf_marginal_accuracy: 0.2231 - val_loss: 0.3944 - val_crf_marginal_accuracy: 0.8941
Epoch 3/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.3438 - crf_marginal_accuracy: 0.9376 - val_loss: 0.2690 - val_crf_marginal_accuracy: 0.9606
Epoch 4/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.2541 - crf_marginal_accuracy: 0.9644 - val_loss: 0.2149 - val_crf_marginal_accuracy: 0.9663
Epoch 5/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.2091 - crf_marginal_accuracy: 0.9677 - val_loss: 0.1809 - val_crf_marginal_accuracy: 0.9699
Epoch 6/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.1821 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.2329    0.5667    0.3301       150
 I-DOCAGRDATE     0.5431    0.7159    0.6176        88
  B-DOCAGRNUM     0.3551    0.6369    0.4560       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.2534    0.7739    0.3818      1243
  I-DOCAMOUNT     0.3079    0.8000    0.4446       620
    B-DOCCPTY     0.5005    0.8672    0.6347       595
    I-DOCCPTY     0.6462    0.8936    0.7500       564
 B-DOCCPTYINN     0.3796    0.9220    0.5377       282
B-DOCCUSTOMER     0.4562    0.8883    0.6028       609
I-DOCCUSTOMER     0.5298    0.9632    0.6836       462
    B-DOCDATE     0.3298    0.8248    0.4712       645
    I-DOCDATE     0.6376    0.9802    0.7726       605
     B-DOCNUM     0.2767    0.6805    0.3934       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.3734    0.8290    0.5149      6700
    macro avg     0.3632    0.7009    0.4718      6700
 weighte

## W2V(not fixed, window=5, min_count=1) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=5, min_count=1)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_7 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 3,450,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 44s 15ms/step - loss: 0.8051 - crf_marginal_accuracy: 0.1195 - val_loss: 0.6141 - val_crf_marginal_accuracy: 0.8147
Epoch 2/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.5369 - crf_marginal_accuracy: 0.8713 - val_loss: 0.3980 - val_crf_marginal_accuracy: 0.9200
Epoch 3/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.3585 - crf_marginal_accuracy: 0.9483 - val_loss: 0.2807 - val_crf_marginal_accuracy: 0.9671
Epoch 4/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.2715 - crf_marginal_accuracy: 0.9674 - val_loss: 0.2284 - val_crf_marginal_accuracy: 0.9650
Epoch 5/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.2207 - crf_marginal_accuracy: 0.9689 - val_loss: 0.1927 - val_crf_marginal_accuracy: 0.9749
Epoch 6/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.1837 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.7090    0.6333    0.6690       150
 I-DOCAGRDATE     0.7416    0.7500    0.7458        88
  B-DOCAGRNUM     0.5857    0.6872    0.6324       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.7106    0.6854    0.6978      1243
  I-DOCAMOUNT     0.7516    0.5710    0.6489       620
    B-DOCCPTY     0.8000    0.9076    0.8504       595
    I-DOCCPTY     0.8257    0.8989    0.8608       564
 B-DOCCPTYINN     0.8810    0.9184    0.8993       282
B-DOCCUSTOMER     0.9165    0.9015    0.9089       609
I-DOCCUSTOMER     0.8459    0.9502    0.8950       462
    B-DOCDATE     0.7926    0.8651    0.8273       645
    I-DOCDATE     0.8428    0.9570    0.8963       605
     B-DOCNUM     0.6849    0.6544    0.6693       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.7844    0.7981    0.7912      6700
    macro avg     0.6725    0.6920    0.6801      6700
 weighte

## W2V(fixed, window=5, min_count=1) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_8 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 124,576
Non-trainable params: 3,325,900
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 43s 15ms/step - loss: 0.7155 - crf_marginal_accuracy: 0.0766 - val_loss: 0.4706 - val_crf_marginal_accuracy: 0.1218
Epoch 2/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.4088 - crf_marginal_accuracy: 0.8213 - val_loss: 0.3148 - val_crf_marginal_accuracy: 0.9399
Epoch 3/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.3034 - crf_marginal_accuracy: 0.9476 - val_loss: 0.2484 - val_crf_marginal_accuracy: 0.9595
Epoch 4/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.2486 - crf_marginal_accuracy: 0.9584 - val_loss: 0.2087 - val_crf_marginal_accuracy: 0.9615
Epoch 5/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.2117 - crf_marginal_accuracy: 0.9630 - val_loss: 0.1809 - val_crf_marginal_accuracy: 0.9632
Epoch 6/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.1870 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.2707    0.6333    0.3792       150
 I-DOCAGRDATE     0.3730    0.5341    0.4393        88
  B-DOCAGRNUM     0.2243    0.6592    0.3348       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.1402    0.8142    0.2392      1243
  I-DOCAMOUNT     0.3796    0.6919    0.4903       620
    B-DOCCPTY     0.4909    0.8605    0.6252       595
    I-DOCCPTY     0.4819    0.8493    0.6149       564
 B-DOCCPTYINN     0.1714    0.9504    0.2904       282
B-DOCCUSTOMER     0.3422    0.8686    0.4910       609
I-DOCCUSTOMER     0.5100    0.9372    0.6606       462
    B-DOCDATE     0.3530    0.7318    0.4763       645
    I-DOCDATE     0.6935    0.9686    0.8083       605
     B-DOCNUM     0.3574    0.6621    0.4642       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.2888    0.8076    0.4255      6700
    macro avg     0.3192    0.6774    0.4209      6700
 weighte

## W2V(not fixed, window=5, max_vocab_size=15000) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=5, max_vocab_size=15000)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 943, 50)           238250    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_9 (CRF)                  (None, 943, 16)           560       
Total params: 362,826
Trainable params: 362,826
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 45s 16ms/step - loss: 0.7793 - crf_marginal_accuracy: 0.2451 - val_loss: 0.5621 - val_crf_marginal_accuracy: 0.0712
Epoch 2/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.5019 - crf_marginal_accuracy: 0.7804 - val_loss: 0.3784 - val_crf_marginal_accuracy: 0.9175
Epoch 3/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.3482 - crf_marginal_accuracy: 0.9454 - val_loss: 0.2692 - val_crf_marginal_accuracy: 0.9551
Epoch 4/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.2617 - crf_marginal_accuracy: 0.9578 - val_loss: 0.2135 - val_crf_marginal_accuracy: 0.9590
Epoch 5/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.2113 - crf_marginal_accuracy: 0.9626 - val_loss: 0.1788 - val_crf_marginal_accuracy: 0.9657
Epoch 6/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.1795 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.2073    0.6067    0.3090       150
 I-DOCAGRDATE     0.5492    0.7614    0.6381        88
  B-DOCAGRNUM     0.2872    0.6145    0.3915       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.1751    0.8423    0.2899      1243
  I-DOCAMOUNT     0.3390    0.6758    0.4515       620
    B-DOCCPTY     0.4278    0.8958    0.5790       595
    I-DOCCPTY     0.5349    0.9096    0.6737       564
 B-DOCCPTYINN     0.3703    0.9113    0.5266       282
B-DOCCUSTOMER     0.6198    0.8966    0.7329       609
I-DOCCUSTOMER     0.7008    0.9329    0.8004       462
    B-DOCDATE     0.3279    0.8744    0.4770       645
    I-DOCDATE     0.7016    0.9636    0.8120       605
     B-DOCNUM     0.2926    0.6774    0.4087       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.3372    0.8361    0.4806      6700
    macro avg     0.3689    0.7042    0.4727      6700
 weighte

## W2V(fixed, window=5, max_vocab_size=15000) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 943, 50)           238250    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 943, 16)           3216      
_________________________________________________________________
crf_10 (CRF)                 (None, 943, 16)           560       
Total params: 362,826
Trainable params: 124,576
Non-trainable params: 238,250
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 45s 16ms/step - loss: 0.7324 - crf_marginal_accuracy: 0.2027 - val_loss: 0.4870 - val_crf_marginal_accuracy: 0.1033
Epoch 2/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.4326 - crf_marginal_accuracy: 0.6717 - val_loss: 0.3303 - val_crf_marginal_accuracy: 0.9370
Epoch 3/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.3191 - crf_marginal_accuracy: 0.9390 - val_loss: 0.2592 - val_crf_marginal_accuracy: 0.9487
Epoch 4/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.2568 - crf_marginal_accuracy: 0.9478 - val_loss: 0.2147 - val_crf_marginal_accuracy: 0.9415
Epoch 5/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.2175 - crf_marginal_accuracy: 0.9505 - val_loss: 0.1879 - val_crf_marginal_accuracy: 0.9582
Epoch 6/30
2880/2880 [==============================] - 36s 13ms/step - loss: 0.1932 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.1492    0.6533    0.2429       150
 I-DOCAGRDATE     0.3315    0.6818    0.4461        88
  B-DOCAGRNUM     0.1864    0.6872    0.2932       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.1455    0.7828    0.2454      1243
  I-DOCAMOUNT     0.1515    0.6145    0.2431       620
    B-DOCCPTY     0.3483    0.8353    0.4916       595
    I-DOCCPTY     0.5185    0.8954    0.6567       564
 B-DOCCPTYINN     0.1888    0.9291    0.3138       282
B-DOCCUSTOMER     0.5268    0.8883    0.6614       609
I-DOCCUSTOMER     0.5321    0.9329    0.6777       462
    B-DOCDATE     0.3143    0.7612    0.4449       645
    I-DOCDATE     0.6949    0.9488    0.8022       605
     B-DOCNUM     0.2883    0.6498    0.3994       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.2655    0.7999    0.3987      6700
    macro avg     0.2917    0.6840    0.3946      6700
 weighte

## W2V(not fixed, window=25) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=25, min_count=2)

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 943, 50)           1240150   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 943, 16)           3216      
_________________________________________________________________
crf_11 (CRF)                 (None, 943, 16)           560       
Total params: 1,364,726
Trainable params: 1,364,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 47s 16ms/step - loss: 0.8186 - crf_marginal_accuracy: 0.4977 - val_loss: 0.6030 - val_crf_marginal_accuracy: 0.8677
Epoch 2/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.5151 - crf_marginal_accuracy: 0.9230 - val_loss: 0.3785 - val_crf_marginal_accuracy: 0.9493
Epoch 3/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.3439 - crf_marginal_accuracy: 0.9535 - val_loss: 0.2713 - val_crf_marginal_accuracy: 0.9643
Epoch 4/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.2601 - crf_marginal_accuracy: 0.9664 - val_loss: 0.2198 - val_crf_marginal_accuracy: 0.9627
Epoch 5/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.2130 - crf_marginal_accuracy: 0.9718 - val_loss: 0.1885 - val_crf_marginal_accuracy: 0.9704
Epoch 6/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.1817 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.7302    0.6133    0.6667       150
 I-DOCAGRDATE     0.7407    0.6818    0.7101        88
  B-DOCAGRNUM     0.7708    0.6201    0.6873       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.8131    0.6372    0.7145      1243
  I-DOCAMOUNT     0.7747    0.5435    0.6389       620
    B-DOCCPTY     0.8921    0.8756    0.8838       595
    I-DOCCPTY     0.8540    0.8918    0.8725       564
 B-DOCCPTYINN     0.9577    0.8830    0.9188       282
B-DOCCUSTOMER     0.9513    0.8982    0.9240       609
I-DOCCUSTOMER     0.9575    0.9264    0.9417       462
    B-DOCDATE     0.8740    0.8171    0.8446       645
    I-DOCDATE     0.8887    0.9372    0.9123       605
     B-DOCNUM     0.8113    0.6406    0.7159       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.8628    0.7688    0.8131      6700
    macro avg     0.7344    0.6644    0.6954      6700
 weighte

## GLOVE (not fixed) + BILSTM + CRF

In [ ]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(df['tokens'], window=15)

glove = Glove(no_components=300, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 943, 300)          19955400  
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_12 (TimeDis (None, 943, 16)           3216      
_________________________________________________________________
crf_12 (CRF)                 (None, 943, 16)           560       
Total params: 20,279,976
Trainable params: 20,279,976
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 48s 17ms/step - loss: 0.9013 - crf_marginal_accuracy: 0.0018 - val_loss: 0.7501 - val_crf_marginal_accuracy: 0.0026
Epoch 2/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.6822 - crf_marginal_accuracy: 0.0039 - val_loss: 0.5428 - val_crf_marginal_accuracy: 0.0201
Epoch 3/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.4689 - crf_marginal_accuracy: 0.5339 - val_loss: 0.3649 - val_crf_marginal_accuracy: 0.9177
Epoch 4/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.3202 - crf_marginal_accuracy: 0.9549 - val_loss: 0.2745 - val_crf_marginal_accuracy: 0.9752
Epoch 5/30
2880/2880 [==============================] - 39s 13ms/step - loss: 0.2322 - crf_marginal_accuracy: 0.9796 - val_loss: 0.2123 - val_crf_marginal_accuracy: 0.9878
Epoch 6/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.1696 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.7252    0.6333    0.6762       150
 I-DOCAGRDATE     0.6542    0.7955    0.7179        88
  B-DOCAGRNUM     0.7925    0.4693    0.5895       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.8304    0.5632    0.6711      1243
  I-DOCAMOUNT     0.7613    0.4629    0.5757       620
    B-DOCCPTY     0.9048    0.8471    0.8750       595
    I-DOCCPTY     0.9241    0.8422    0.8813       564
 B-DOCCPTYINN     0.9673    0.8404    0.8994       282
B-DOCCUSTOMER     0.9572    0.9540    0.9556       609
I-DOCCUSTOMER     0.9236    0.9416    0.9325       462
    B-DOCDATE     0.8804    0.8217    0.8500       645
    I-DOCDATE     0.8640    0.9455    0.9029       605
     B-DOCNUM     0.8467    0.6022    0.7038       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.8728    0.7406    0.8013      6700
    macro avg     0.7354    0.6479    0.6821      6700
 weighte

## GLOVE (fixed) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 943, 300)          19955400  
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_13 (TimeDis (None, 943, 16)           3216      
_________________________________________________________________
crf_13 (CRF)                 (None, 943, 16)           560       
Total params: 20,279,976
Trainable params: 324,576
Non-trainable params: 19,955,400
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 48s 17ms/step - loss: 0.8907 - crf_marginal_accuracy: 0.3582 - val_loss: 0.7714 - val_crf_marginal_accuracy: 0.8151
Epoch 2/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.7298 - crf_marginal_accuracy: 0.1789 - val_loss: 0.6436 - val_crf_marginal_accuracy: 0.1215
Epoch 3/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.6107 - crf_marginal_accuracy: 0.1124 - val_loss: 0.5278 - val_crf_marginal_accuracy: 0.1353
Epoch 4/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.5076 - crf_marginal_accuracy: 0.1212 - val_loss: 0.4371 - val_crf_marginal_accuracy: 0.2811
Epoch 5/30
2880/2880 [==============================] - 38s 13ms/step - loss: 0.4291 - crf_marginal_accuracy: 0.8404 - val_loss: 0.3742 - val_crf_marginal_accuracy: 0.9145
Epoch 6/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.3698 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

 B-DOCAGRDATE     0.0000    0.0000    0.0000       150
 I-DOCAGRDATE     0.0000    0.0000    0.0000        88
  B-DOCAGRNUM     0.0563    0.0894    0.0691       179
  I-DOCAGRNUM     0.0000    0.0000    0.0000         7
  B-DOCAMOUNT     0.0366    0.7401    0.0697      1243
  I-DOCAMOUNT     0.0549    0.0855    0.0669       620
    B-DOCCPTY     0.2047    0.7143    0.3182       595
    I-DOCCPTY     0.4160    0.4965    0.4527       564
 B-DOCCPTYINN     0.1605    0.6631    0.2585       282
B-DOCCUSTOMER     0.2979    0.6125    0.4009       609
I-DOCCUSTOMER     0.3786    0.7056    0.4928       462
    B-DOCDATE     0.1041    0.5953    0.1772       645
    I-DOCDATE     0.4078    0.7091    0.5178       605
     B-DOCNUM     0.1888    0.5975    0.2870       651
     I-DOCNUM     0.0000    0.0000    0.0000         0

    micro avg     0.0964    0.5645    0.1647      6700
    macro avg     0.1538    0.4006    0.2074      6700
 weighte